In [379]:
df

,departureCity,arrivalCity,Distance,Demand
0,Paris,Madrid,654,713
1,Amsterdam,Madrid,920,660
2,Paris,Istanbul,1404,652
3,Madrid,Paris,654,651
4,London,Manchester,163,649
5,Frankfurt,Madrid,899,643
6,Moscow,Amsterdam,1338,633
7,London,Dublin,288,629
8,Amsterdam,Moscow,1338,629


In [619]:
import gurobipy as gp
import numpy as np
import pandas as pd

df = pd.read_csv("C:/Users/mauri/Documents/Master/BAM/Management Science/Group assignment/flights.csv")
flights=[]
for index,row in df.iterrows():
    flights.append((row.departureCity,row.arrivalCity))
    flights.append((row.departureCity,row.arrivalCity))
flights=list(set(flights))#removing duplicates

aeroplaneSizes={'small': 50, 'medium': 100, 'large': 300}
aeroplaneCost={'small': 4.5, 'medium': 8, 'large': 20}
Fares = {}
#Winst = (0.1 * aantal kilometer vlucht * aantal passagiers) - (vliegkosten * kilometer)
Demand = {}
Distance = {}
for index,row in df.iterrows():
    Distance[row.departureCity,row.arrivalCity]=int(row.Distance)
    Demand[row.departureCity,row.arrivalCity]=int(row.Demand)

Planes={}
Tickets={}
SPlanes={}
MPlanes={}
LPlanes={}
Fare={}
for index,row in df.iterrows():
    l = 0
    m = 0
    s = 0
    t = 0
    nd = row.Demand
    while nd > 199:
        if nd / aeroplaneSizes.get('large') > 0.66:
            l = l + 1
            nd = nd - 300
            t = t + 300
    Planes[row.departureCity,row.arrivalCity,"large"]= l
    while nd > 79:
        if nd / aeroplaneSizes.get('medium') > 0.79:
            m = m + 1
            nd = nd - 100
            t = t + 100
    Planes[row.departureCity,row.arrivalCity,"medium"] = m
    while nd > 44:
        if nd / aeroplaneSizes.get('small') > 0.89:
            s = s + 1
            nd = nd - 50
            t = t + 50
    Planes[row.departureCity,row.arrivalCity,"small"]= s
    Planes[row.departureCity,row.arrivalCity,"tickets"]=t
    Tickets[row.departureCity,row.arrivalCity]=int(t)    
    SPlanes[row.departureCity,row.arrivalCity]= s
    MPlanes[row.departureCity,row.arrivalCity]= m
    LPlanes[row.departureCity,row.arrivalCity]= l

Profit={}
for t in Tickets:
    profit = ((
                    ((Tickets.get(t) * 0.1 * Distance.get(t)) 
                      - 
                      (SPlanes.get(t) * 4.5 * Distance.get(t) + MPlanes.get(t) * 8 * Distance.get(t) + MPlanes.get(t) * 20 * Distance.get(t)))
                   ))
    Profit[t]=profit

In [620]:
model=gp.Model()
x=model.addVars(Fare,ub=Demand,obj=Profit)
model.modelSense=gp.GRB.MAXIMIZE

#Cons?

print(Planes)
model.optimize()

print(sum(Profit.values()))

{('Paris', 'Madrid', 'large'): 2, ('Paris', 'Madrid', 'medium'): 1, ('Paris', 'Madrid', 'small'): 0, ('Paris', 'Madrid', 'tickets'): 700, ('Amsterdam', 'Madrid', 'large'): 2, ('Amsterdam', 'Madrid', 'medium'): 0, ('Amsterdam', 'Madrid', 'small'): 1, ('Amsterdam', 'Madrid', 'tickets'): 650, ('Paris', 'Istanbul', 'large'): 2, ('Paris', 'Istanbul', 'medium'): 0, ('Paris', 'Istanbul', 'small'): 1, ('Paris', 'Istanbul', 'tickets'): 650, ('Madrid', 'Paris', 'large'): 2, ('Madrid', 'Paris', 'medium'): 0, ('Madrid', 'Paris', 'small'): 1, ('Madrid', 'Paris', 'tickets'): 650, ('London', 'Manchester', 'large'): 2, ('London', 'Manchester', 'medium'): 0, ('London', 'Manchester', 'small'): 1, ('London', 'Manchester', 'tickets'): 650, ('Frankfurt', 'Madrid', 'large'): 2, ('Frankfurt', 'Madrid', 'medium'): 0, ('Frankfurt', 'Madrid', 'small'): 0, ('Frankfurt', 'Madrid', 'tickets'): 600, ('Moscow', 'Amsterdam', 'large'): 2, ('Moscow', 'Amsterdam', 'medium'): 0, ('Moscow', 'Amsterdam', 'small'): 0, ('Mos